In [2]:
import json
import os
from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv

In [5]:
#PET0 

csv_pet0_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_CSV_DATA/Export_Gained/data.csv'

with open(csv_pet0_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne


In [7]:
csv_data[1]

['alina',
 '11011101021002',
 '0',
 '18',
 '1.2.250.1.59.470.940100027.1.20130128113103.7763.17688',
 '2013-01-30',
 '0',
 '0',
 '0',
 'review18.csv',
 '4',
 '29.35',
 'Abdominal',
 '13.6',
 'Abdominal',
 '1.35',
 '1.17',
 '0.1',
 '1.84',
 '1.49',
 '0.11',
 '1.99',
 '1.52',
 '0.17',
 '0',
 '12',
 '3.17',
 '2.36',
 '0.49',
 '2',
 '2',
 '19',
 '1.2.840.113704.1.111.2180.1361963756.13',
 '2013-02-27',
 '0',
 '0',
 '0',
 'review19.csv',
 '1.96',
 'Intestine - small and colon',
 '1.53',
 '1.4',
 '0.12',
 '2.27',
 '1.86',
 '0.16',
 '1',
 '0',
 '0',
 '4',
 '20',
 '1.2.840.113704.1.111.4636.1364288605.3',
 '2013-03-26',
 '0',
 '0',
 '0',
 'review20.csv',
 '1.96',
 'Intestine - small and colon',
 '1.53',
 '1.4',
 '0.12',
 '2.27',
 '1.86',
 '0.16',
 '1',
 '0',
 '0']

In [16]:
nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET2'
liste_nifti = os.listdir(nifti_directory)

In [17]:
liste = []
for row in csv_data : 
    subliste = []
    #subliste.append(row[0])
    subliste.append(row[1])
    subliste.append(row[32])
    study_uid = row[32]
    subsubliste = []
    for nifti in liste_nifti : 
        if study_uid in nifti: 
            subsubliste.append(nifti)
    if len(subsubliste) == 2 : 
        subsubliste = sorted(subsubliste)
        #print(subsubliste)
        subliste.append(subsubliste[0])
        subliste.append(subsubliste[1])
        #subliste.append(subsubliste[2])
    else : 
        pass
    subliste.append("pet2")
    liste.append(subliste)



In [18]:
len(liste)

728

In [19]:
liste[10]

['11011101021011',
 '1.2.250.1.59.470.940100027.1.20140522113117.1243.111672',
 '1.2.250.1.59.470.940100027.1.20140522113117.1243.111672_nifti_CT.nii',
 '1.2.250.1.59.470.940100027.1.20140522113117.1243.111672_nifti_PT.nii',
 'pet2']

In [20]:
#clean -> if len != 6 
#doit y avoir 573
completed_liste = []
for serie in liste : 
    if len(serie) == 5 : 
        completed_liste.append(serie)


In [21]:
len(completed_liste)

595

In [22]:
new_liste = []
new_liste.append(completed_liste[0])
for i in range(1, len(completed_liste)) :
    if completed_liste[i] not in new_liste : 
        new_liste.append(completed_liste[i])

In [23]:
len(new_liste)

536

In [24]:
new_liste[10]

['11011101021014',
 '1.2.840.113704.1.111.3608.1434978016.23',
 '1.2.840.113704.1.111.3608.1434978016.23_nifti_CT.nii',
 '1.2.840.113704.1.111.3608.1434978016.23_nifti_PT.nii',
 'pet2']

In [25]:
import shutil

In [30]:
#creation sous dossier patient id -> pet0 -> mask ct pt 

for serie in new_liste: 
    patient_id = serie[0]
    pet_type = serie[-1]
    nifti_ct = serie[2]
    nifti_pet = serie[3]
    #nifti_mask = serie[5]

    folder = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI'+'/'+patient_id

    if not os.path.exists(folder):
        os.makedirs(folder)
        print(folder, 'created')
    else : print(folder, "Dossier patient déjà crée")

    subfolder = folder+'/'+pet_type
    if not os.path.exists(subfolder) :
        os.makedirs(subfolder)
        print(subfolder, 'created')
    else : print(subfolder, "déjà crée => prblm")

    ct_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET2'+'/'+nifti_ct
    pt_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI_PET2'+'/'+nifti_pet
    #mask_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'+'/'+nifti_mask

    new_ct_path = subfolder+'/'+nifti_ct
    new_pet_path = subfolder+'/'+nifti_pet
    #new_mask_path = subfolder+'/'+nifti_mask

    os.rename(ct_path, subfolder+'/'+nifti_ct)
    os.rename(pt_path, subfolder+'/'+nifti_pet)
    #os.rename(mask_path, subfolder+'/'+nifti_mask)
    try : 
        shutil.rmtree('/media/deeplearning/Elements/GAINED_Validated_DICOMS'+'/'+patient_id+'/'+study_uid)
    except Exception as err : 
        print(serie)
        print(err)



    



r patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/51011101881010/pet2 created
['51011101881010', '1.2.250.1.199.10.90001.91642.20131206121722', '1.2.250.1.199.10.90001.91642.20131206121722_nifti_CT.nii', '1.2.250.1.199.10.90001.91642.20131206121722_nifti_PT.nii', 'pet2']
[Errno 2] No such file or directory: '/media/deeplearning/Elements/GAINED_Validated_DICOMS/51011101881010/1.2.840.113619.2.55.3.279710477.416.1398319155.499'
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/51011101881011 Dossier patient déjà crée
/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/GAINED_NIFTI/51011101881011/pet2 created
['51011101881011', '1.2.250.1.199.10.90001.91745.20131226085649', '1.2.250.1.199.10.90001.91745.20131226085649_nifti_CT.nii', '1.2.250.1.199.10.90001.91745.20131226085649_nifti_PT.nii', 'pet2']
[Errno 2] No such file or directory: '/media/deeplearning/Elements/GAINED_Validated_DICOMS/51011101881011/1.2.840.113619.2.55.3

In [18]:
json_path = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI/json/dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

63


In [27]:
for serie in dataset :
    if 'CR 097' in serie[0] : 
        print(serie)

['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Derniers patients/2017097 CR 097/A20080862074 CORPS ENTIER AU FDG/CT CT TAP 55 B30s', 'CT', '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Derniers patients/2017097 CR 097/A20080862074 CORPS ENTIER AU FDG/PT PET TAP AC HD', 'PT', '2.16.840.1.113669.632.20.310000.20001712333', '2017097', 'cr_097_dec 22_2009.csv', 'cr_097_dec 7_2011.csv']
['/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Derniers patients/2017097 CR 097/A21386336556 TEP corps entier/PT PET TAP AC HD', 'PT', '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP/DICOM/Derniers patients/2017097 CR 097/A21386336556 TEP corps entier/CT CT TAP 3.0 B30s', 'CT', '2.16.840.1.113669.632.20.310000.20002475031', '2017097', 'cr_097_dec 7_2011.csv']


In [17]:
for serie in new_liste : 
    if 'CR,002' in serie : 
        print(serie)

['CR,002', '2017002', '1.2.840.113711.41813.3.12152.423471272.26.2116281012.135370', '1.2.840.113711.41813.3.12152.423471272.26.2116281012.135370_nifti_CT.nii', '1.2.840.113711.41813.3.12152.423471272.26.2116281012.135370_nifti_PT.nii', '1.2.840.113711.41813.3.12152.423471272.26.2116281012.135370_nifti_mask.nii', 'pet0']
['CR,002', '2017002', '1.3.51.0.1.1.10.10.13.63.102986123.4600889', '1.3.51.0.1.1.10.10.13.63.102986123.4600889_nifti_CT.nii', '1.3.51.0.1.1.10.10.13.63.102986123.4600889_nifti_PT.nii', '1.3.51.0.1.1.10.10.13.63.102986123.4600889_nifti_mask.nii', 'pet0']
['CR,002', '2017002', '2.16.840.1.113669.632.20.310000.20002733290', '2.16.840.1.113669.632.20.310000.20002733290_nifti_CT.nii', '2.16.840.1.113669.632.20.310000.20002733290_nifti_PT.nii', '2.16.840.1.113669.632.20.310000.20002733290_nifti_mask.nii', 'pet0']


In [16]:
nifti_directory = '/media/deeplearning/78ca2911-9e9f-4f78-b80a-848024b95f92/FLIP_NIFTI'

with open(os.path.join(nifti_directory, 'FLIP_PET0_NIFTI.csv'), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT_NAME", "PATIENT ID", "STUDY UID", "PET", "NIFTI_CT", "NIFTI_PET", "NIFTI_MASK"])
    for serie in new_liste: 
        csv_writer.writerow([serie[0], serie[1], serie[2], serie[-1], os.path.join(nifti_directory+'/'+serie[0]+'/'+serie[-1]+'/'+serie[3]), os.path.join(nifti_directory+'/'+serie[0]+'/'+serie[-1]+'/'+serie[4]), os.path.join(nifti_directory+'/'+serie[0]+'/'+serie[-1]+'/'+serie[5])])


In [ ]:
#REUNITE CSV 
